In [93]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import shutil

In [94]:
#Obtenemos el nombre de las carpetas existentes en la carpeta faces
carpetas = os.listdir('faces')

In [95]:
#Creamos un array donde se guardara un 1 si la imagen tiene gafas y un 0 si no las tiene
gafas = []

In [96]:
#Creamos el directorio donde se guardaran las nuevas imagenes
os.makedirs('imagenes', exist_ok=True)

In [97]:
#Iteramos sobre las carpetas y extraemos las imagenes que no tengan gafas y las que si, ademas de guardar el valor en el array gafas
cont = 0
for i in carpetas:
  archivo_carpeta = os.listdir(f'faces/{i}')
  for archivos in archivo_carpeta:
    if not re.search(r'(2|4)\.pgm$', archivos):
      if re.search(r'sunglasses.pgm$', archivos):
        shutil.copy(f'faces/{i}/{archivos}', f'imagenes/imagen_{cont}.pgm')
        gafas.append(1)
      elif re.search(r'\.bad$', archivos):
        continue
      else:
        shutil.copy(f'faces/{i}/{archivos}', f'imagenes/imagen_{cont}.pgm')
        gafas.append(0)
      cont += 1

In [118]:
#Creamos el dataframe con los datos obtenidos
df = pd.DataFrame({'Image': os.listdir('imagenes'), 'Sunglasses': gafas})

In [126]:
#Guardamos el dataframe en un csv
df.to_csv('data_imagenes.csv', index=False)

In [125]:
df.sample(10).values

array([['imagen_77.pgm', 1],
       ['imagen_480.pgm', 0],
       ['imagen_388.pgm', 0],
       ['imagen_383.pgm', 1],
       ['imagen_415.pgm', 0],
       ['imagen_473.pgm', 0],
       ['imagen_242.pgm', 0],
       ['imagen_325.pgm', 0],
       ['imagen_331.pgm', 1],
       ['imagen_622.pgm', 0]], dtype=object)